In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

from tqdm.notebook import tqdm
from tqdm import tqdm
from tqdm import trange

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.metrics import recall_score
from sklearn.metrics import ConfusionMatrixDisplay

import random

In [2]:
from AgentBasedModel import *
from AgentBasedModel.extra import *
from AgentBasedModel.visualization import *
from AgentBasedModel.visualization.other import plot_book
from AgentBasedModel.visualization.other import plot_full_book
from AgentBasedModel.visualization.trader import *
from random import randint

In [3]:
run utils_nikita//functions.ipynb

# Simulation function

In [4]:
def simulation_with_lob(simulator, exchange_id, n, limit=15, random_state=None, 
                         bid_ask_volume_imbalance_window = [1, 3, 5, 10], sign_transaction_volume_window = [1, 5, 10, 20],
                         class_division=False):
    
    feature_list = []
    lob_data = []
        
    if random_state:
        random.seed(random_state)
    
    for i in tqdm(range(n)):
        
        feature_list.append([])
        
        info = simulator.info
        
#         random.seed(random_state)
#         print(random.random())
        
        simulator.simulate(1, silent=True)
        
        col_names = []
        
        for w in bid_ask_volume_imbalance_window:
            
            bid_ask_volume_imbalance_w = get_bid_ask_volume_imbalance(info, exchange_id, depth=w, class_division=class_division)
            
            if class_division:
                
                present_classes = set(list(map(lambda x: x.type, simulator.traders)))
                current_keys = bid_ask_volume_imbalance_w.keys()
                
                for cls in present_classes:
                    
                    if cls in current_keys:
                        feature_list[i].append(bid_ask_volume_imbalance_w[cls])
                        col_names.append(f"bid_ask_volume_imbalance_{cls.replace(' ', '_')}_{w}")
                    else:
                        feature_list[i].append(0)
                        col_names.append(f"bid_ask_volume_imbalance_{cls.replace(' ', '_')}_{w}")                        
            
            else:
                feature_list[i].append(bid_ask_volume_imbalance_w)
                col_names.append(f"bid_ask_volume_imbalance_{w}")
        
        for w in sign_transaction_volume_window:
            
            sign_transaction_volume_w = get_transaction_volume(info, exchange_id, limit=w)
            
            feature_list[i].append(sign_transaction_volume_w)
            col_names.append(f"sign_transaction_volume_{w}")
        
        feature_list[i].append(get_all_transaction_volume(info, exchange_id, limit=1))
        col_names.append("all_transaction_volume_1")

        # print(simulator.exchanges[0], exchange_id)
        
        ask_data = pd.DataFrame(info.exchanges[exchange_id].order_book['ask'].to_list())[['price', 'qty']]
        bid_data = pd.DataFrame(info.exchanges[exchange_id].order_book['bid'].to_list())[['price', 'qty']]
        
        lob_data.append({'ask': ask_data, 'bid': bid_data})
#         print(lob_data[-1]['ask'].to_list()[0])
#         print(len(lob_data[-1]['ask']))
        
#     print(col_names)
    result_df = pd.DataFrame(feature_list, columns=col_names)

        
    bid_ask_spread = get_spread(info, exchange_id)
    
    last_vals = len(bid_ask_spread)
    
    bid_ask_spread = bid_ask_spread[last_vals - n:]
    
    result_df['bid_ask_spread'] = bid_ask_spread
    result_df['dividends'] = info.dividends[exchange_id][last_vals - n:] 
    result_df['price'] = info.prices[exchange_id][last_vals - n:]
    result_df['dividends_previous_divided'] = (result_df['dividends'] / (result_df['price'] * simulator.exchanges[0].risk_free_rate).shift(1))
    
    fundamental_value_data = info.fundamental_value(exchange_id)
    n_past_iterations = len(fundamental_value_data)
    
    result_df['fundamental_v'] = fundamental_value_data[n_past_iterations - n:]
    
    result_df['return_1'] = ((result_df['price'] - result_df['price'].shift(1))/result_df['price']) #*100
    result_df['return_5'] = ((result_df['price'] - result_df['price'].shift(5))/result_df['price']) #*100
    result_df['return_10'] = ((result_df['price'] - result_df['price'].shift(10))/result_df['price']) #*100
    result_df['return_20'] = ((result_df['price'] - result_df['price'].shift(20))/result_df['price']) #*100
    
    return result_df, lob_data


def make_many_simulations_lob(n_sim=5, n_iter=100, risk_free_rate=0.05, price=100, 
                          dividend=None, random_state=None, silent=False, market_agents=[6, 6, 6, 1],
                          class_division=False, sim_return=False, mm_softlimit=100):
    
    if not dividend:
        dividend = price * risk_free_rate
    
    risk_free_rate = risk_free_rate
    price = price
    dividend = dividend
    
    resulting_df = []
    resulting_lob = []
    
    simulators=[]
    
    for i in range(n_sim):
        
        random.seed(random_state)
        
        # print(random_state)
    
        assets = [Stock(dividend)]

        simple_exchange = ExchangeAgent(assets[0], risk_free_rate, mean=price)
        
        simple_traders = [
            *[Random(simple_exchange) for _ in range(market_agents[0])],
            *[Chartist1D(simple_exchange) for _ in range(market_agents[1])],
            *[Fundamentalist(simple_exchange) for _ in range(market_agents[2])],
            *[MarketMaker1D(simple_exchange, softlimit = mm_softlimit) for _ in range(market_agents[3])]
        ]
        
        # *[Universalist(market=simple_exchange) for _ in range(100)],

        simple_sim = Simulator(**{
            'assets': [assets[0]],
            'exchanges': [simple_exchange],
            'traders': simple_traders
        })
        
        exchange_id = simple_exchange.id
        
        simulators.append(simple_sim)
        
        # bug report
#         if i == 0:
#             features_simulation_data = simulation_with_data(simple_sim, exchange_id, n_iter, random_state=None)
#             make_graphs(features_simulation_data, features_simulation_data.columns)
#             continue
        
#         try:

        features_simulation_data, lob_data = simulation_with_lob(simple_sim, exchange_id, n_iter, random_state=None, class_division=class_division)
        print(f"--step {i + 1} of simulation results with random state {random_state}--")

        if not silent:
            make_graphs(features_simulation_data, features_simulation_data.columns)

        resulting_df.append(features_simulation_data)
        resulting_lob.append(lob_data)
        
        if random_state:
            random_state = (random_state + (i + 1 + (i + 2)**2) + 100 * (i + 3)) % 11221
        
#         except:
#             print('no orders bla bla')
#             print(random_state)
#             return
#     plot_book(simple_sim.info, exchange_id)
    
    if sim_return:
        return resulting_df, simulators, resulting_lob
    else:
        return resulting_df, resulting_lob


# Data generation

In [5]:
random_state_for_simulations = 999 + 21 + 14

market_agents=[40, 40, 40, 0]

In [6]:
huge_data_features, huge_data_lob = make_many_simulations_lob(n_sim=500, n_iter=250, risk_free_rate=0.05, price=200, dividend=10, random_state=random_state_for_simulations, silent=True, market_agents=market_agents)

100%|██████████| 250/250 [00:09<00:00, 25.83it/s]


--step 1 of simulation results with random state 1034--


100%|██████████| 250/250 [00:08<00:00, 28.89it/s]


--step 2 of simulation results with random state 1339--


100%|██████████| 250/250 [00:09<00:00, 26.65it/s]


...

--step 498 of simulation results with random state 8993--


100%|██████████| 250/250 [00:08<00:00, 29.50it/s]


--step 499 of simulation results with random state 5525--


100%|██████████| 250/250 [00:08<00:00, 28.28it/s]

--step 500 of simulation results with random state 3157--


In [7]:
# huge_data_features, huge_data_lob = make_many_simulations_lob(n_sim=2, n_iter=250, risk_free_rate=0.05, price=200, dividend=10, random_state=random_state_for_simulations, silent=True, market_agents=market_agents)

In [8]:
pd.DataFrame(huge_data_lob[0][0]['ask'][:10])

,price,qty
0,200.1,1
1,200.5,1
2,200.5,1
3,200.6,1
4,200.7,2
5,200.7,5
6,200.8,2
7,200.9,4
8,201.1,1
9,201.1,4


In [9]:
pd.DataFrame(huge_data_lob[1][201]['ask'][:100])

,price,qty
0,209.5,3
1,209.5,3
2,209.5,5
3,209.6,1
4,209.6,2
...,...,...
95,212.8,2
96,212.9,5
97,213.0,3
98,213.1,5


In [10]:
def get_bid_ask_huge_lob(lob_data, limit=20):

    for i in range(len(lob_data)):
        
        for j in range(len(lob_data[0])):
            
            lob_data[i][j]['ask'] = lob_data[i][j]['ask'].groupby(['price']).sum().reset_index()
            lob_data[i][j]['bid'] = lob_data[i][j]['bid'].groupby(['price']).sum().reset_index()
        
    return lob_data

In [11]:
len(huge_data_lob) # 200 x 250 x 2

500

In [12]:
get_bid_ask_huge_lob(huge_data_lob);

# Saving lob data

In [14]:
result_df = pd.DataFrame()
    
for smol_d in huge_data_features:
    result_df = pd.concat([result_df, smol_d])
        
result_df.to_csv('data/huge_data_lob_no_mm/huge_data_features.csv')

In [ ]:
sim_n = 0
iter_n = 0

limit=20

for sim_d in huge_data_lob:
    
    iter_n = 0
    
    for iter_lob in sim_d:
        iter_lob['ask'][:limit].to_csv(f'data/huge_data_lob_no_mm/asks/sim_{sim_n}_iter_{iter_n}.csv')
        iter_lob['bid'][-limit:].to_csv(f'data/huge_data_lob_no_mm/bids/sim_{sim_n}_iter_{iter_n}.csv')

        iter_n += 1
    sim_n += 1

In [ ]:
# sim_n = 0
# iter_n = 0

# for sim_d in huge_data_ask:
    
#     iter_n = 0
    
#     for iter_lob in sim_d:
#         iter_lob.to_csv(f'data/huge_data_lob/asks/sim_{sim_n}_iter_{iter_n}.csv')

#         iter_n += 1
#     sim_n += 1

# Downloading lob data

In [ ]:
sim_n = 500
iter_n = 250

huge_data_bid = []
huge_data_ask = []

for i in range(sim_n):

    bids_simulation = []
    asks_simulation = []
    
    for j in range(iter_n):
        cur_bid = pd.read_csv(f'data/huge_data_lob/bids/sim_{i}_iter_{j}.csv')
        cur_ask = pd.read_csv(f'data/huge_data_lob/asks/sim_{i}_iter_{j}.csv')

        bids_simulation.append(cur_bid)
        asks_simulation.append(cur_ask)

    huge_data_bid.append(bids_simulation)
    huge_data_ask.append(asks_simulation)

In [ ]:
huge_data_features = pd.read_csv('data/huge_data_lob/huge_data_features.csv')

In [ ]:
huge_data_bid[0][10]

In [ ]:
huge_data_ask[0][10]